In [3]:
import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score
from datetime import datetime, timedelta
import pytz

API_KEY = '5ce8f928baa39eb5a373e7ffcc0f06d1'
BASE_URL = 'https://api.openweathermap.org/data/2.5/'

def get_current_weather(city):
    url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()
    return {
        'city': data['name'],
        'current_temp': data['main']['temp'],
        'feels_like': round(data['main']['feels_like']),
        'temp_min': data['main']['temp_min'],
        'temp_max': round(data['main']['temp_max']),
        'humidity': data['main']['humidity'],
        'description': data['weather'][0]['description'],
        'country': data['sys']['country'],
        'wind_gust_dir': data['wind']['deg'],
        'wind_gust_speed': data['wind']['speed'],
        'pressure': data['main']['pressure']
    }

def read_historical_data(filename):
    df = pd.read_csv(filename)
    df = df.dropna()
    df = df.drop_duplicates()
    return df

def prepare_data(data):
    le = LabelEncoder()
    data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
    data['RainTomorrow'] = data['RainTomorrow'].map({'Yes': 1, 'No': 0})

    x = data[['MinTemp', 'MaxTemp', 'WindGustDir', 'WindGustSpeed', 'Humidity', 'Pressure', 'Temp']]
    y = data['RainTomorrow']

    return x, y, le

def train_rain_model(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Rain Prediction Model Accuracy: {accuracy * 100:.2f}%")
    return model

def train_regression_model(x, y):
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(x, y)
    return model

def predict_future(model, current_value):
    predictions = [current_value]
    for _ in range(5):
        next_value = model.predict(np.array([[predictions[-1]]]))[0]
        predictions.append(next_value)
    return predictions[1:]

def classify_weather(rain_prediction, description):
    if rain_prediction == 1:
        return "Rainy"
    elif "cloud" in description.lower():
        return "Mostly Cloudy"
    elif "clear" in description.lower():
        return "Sunny"
    else:
        return "Partly Cloudy"

def weather_view():
    city = input('Enter any City name: ')
    current_weather = get_current_weather(city)
    historical_data = read_historical_data('weather.csv')
    x, y, le = prepare_data(historical_data)
    rain_model = train_rain_model(x, y)

    current_data = {
        'MinTemp': current_weather['temp_min'],
        'MaxTemp': current_weather['temp_max'],
        'WindGustDir': 0,  # Default encoding
        'WindGustSpeed': current_weather['wind_gust_speed'],
        'Humidity': current_weather['humidity'],
        'Pressure': current_weather['pressure'],
        'Temp': current_weather['current_temp']
    }

    current_df = pd.DataFrame([current_data])
    rain_prediction = rain_model.predict(current_df)[0]
    weather_condition = classify_weather(rain_prediction, current_weather['description'])

    x_temp, y_temp = historical_data['Temp'][:-1], historical_data['Temp'][1:]
    x_hum, y_hum = historical_data['Humidity'][:-1], historical_data['Humidity'][1:]

    temp_model = train_regression_model(np.array(x_temp).reshape(-1, 1), np.array(y_temp))
    hum_model = train_regression_model(np.array(x_hum).reshape(-1, 1), np.array(y_hum))

    future_temp = predict_future(temp_model, current_weather['current_temp'])
    future_humidity = predict_future(hum_model, current_weather['humidity'])

    timezone = pytz.timezone('Asia/Kolkata')
    current_time = datetime.now(timezone)
    next_hour = (current_time + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
    future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

    print(f"City: {city}, {current_weather['country']}")
    print(f"Current Temperature: {current_weather['current_temp']}°C")
    print(f"Feels Like: {current_weather['feels_like']}°C")
    print(f"Minimum Temperature: {current_weather['temp_min']}°C")
    print(f"Maximum Temperature: {current_weather['temp_max']}°C")
    print(f"Humidity: {current_weather['humidity']}%")
    print(f"Weather Prediction: {weather_condition}")
    print(f"Rain Prediction: {'Yes' if rain_prediction else 'No'}")

    print("\nFuture Temperature Predictions:")
    for time, temp in zip(future_times, future_temp):
        print(f"{time}: {temp:.1f}°C")

    print("\nFuture Humidity Predictions:")
    for time, hum in zip(future_times, future_humidity):
        print(f"{time}: {hum:.1f}%")

weather_view()


Enter any City name: mumbai
Rain Prediction Model Accuracy: 84.93%
City: mumbai, IN
Current Temperature: 28.99°C
Feels Like: 33°C
Minimum Temperature: 28.99°C
Maximum Temperature: 29°C
Humidity: 74%
Weather Prediction: Rainy
Rain Prediction: Yes

Future Temperature Predictions:
23:00: 22.1°C
00:00: 23.3°C
01:00: 19.0°C
02:00: 24.5°C
03:00: 24.5°C

Future Humidity Predictions:
23:00: 44.8%
00:00: 46.2%
01:00: 48.2%
02:00: 53.7%
03:00: 58.1%
